![PPGI_UFRJ](https://github.com/zavaleta/Fundamentos_DS/blob/main/imagens/ppgi-ufrj.png?raw=1)
# Fundamentos de Ciência de Dados
## 2022 - BL2
### **Aluno:** Gilberto Gil, Saulo Andrade Almeida, Valquire Jesus

# Trabalho Final
---

In [129]:
!pip install conda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [130]:
!pip install prov

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [131]:
#checking version machine architecture, OS, python and all libs used in this notebook
import platform as platform
import numpy as np
import pandas as pd
import os
import conda
import pydot
import prov

def checkingEnvironmentVersions(details=False):

    #definnig version of python and all libs used
    HOST_MACHINE_ARCHTECTURE_EXPECTED = 'x86_64'
    HOST_MACHINE_OS_EXPECTED = 'Linux'
    HOST_MACHINE_PLATFORM = 'Linux-5.15.0-47-generic-x86_64-with-glibc2.31'
    CONDA_VERSION_EXPECTED = '4.14.0'
    CONDA_DEFAULT_ENV_EXPECTED = 'drecvmenv'
    PYTHON_VERSION_EXPECTED = '3.9.12'
    NUMPY_LIB_VERSION_EXPECTED = '1.21.5'
    PANDAS_LIB_VERSION_EXPECTED = '1.4.2'
    PYDOT_LIB_VERSION_EXPECTED = '1.4.2'
    PROV_LIB_VERSION_EXPECTED = '2.0.0'
    

    if details: 
        print('Host Machine Architecture:', platform.machine())
        print('Host Machine OS:', platform.system())
        print('Conda Version:', conda.__version__)
        print('Conda default env:', os.environ['CONDA_DEFAULT_ENV'])
        print('Python Version:', platform.python_version())
        print('NumPy Lib Version:', np.__version__)
        print('Pandas Lib Version:', pd.__version__)
        print('PyDot Lib Version:', pydot.__version__)
        print('Prov Lib Version:', prov.__version__)
        
    #checking versions
    try:
        #checking Machine Architecute expected
        assert platform.machine() == HOST_MACHINE_ARCHTECTURE_EXPECTED

        #checking OS expected
        assert platform.system() == HOST_MACHINE_OS_EXPECTED
        
        #checking conda version
        assert conda.__version__ == CONDA_VERSION_EXPECTED
        
        #checking conda default environment
        assert os.environ['CONDA_DEFAULT_ENV'] == CONDA_DEFAULT_ENV_EXPECTED    

        #checking python version
        assert platform.python_version() == PYTHON_VERSION_EXPECTED

        #checking numpy lib version
        assert np.__version__ == NUMPY_LIB_VERSION_EXPECTED  

        #checking Pandas lib version
        assert pd.__version__ == PANDAS_LIB_VERSION_EXPECTED
        
        #checking pydot version
        assert pydot.__version__ == PYDOT_LIB_VERSION_EXPECTED
        
        #checking prov version
        assert prov.__version__ == PROV_LIB_VERSION_EXPECTED
    except:
        #if any assert fail, or something else get wrong during verification
        if details: print('Something is wrong!')
        return False
    else:
        #if pass all asserts
        if details: print('All versions are correct!')
        return True

In [132]:
#Utilizado ambiente python 3.9
import pandas as pd
import numpy as np
#from datetime import datetime
#dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

def loadCsv(path, file):
  if path == 'data/cad-emp/':  # condição para fazer leitura do csv cadastro de companias
    return pd.read_csv(path+file, sep=';', header=0, encoding="ISO-8859-1")
  return pd.read_csv(path+file, sep=';', header=0, encoding="ISO-8859-1", parse_dates=['DT_REFER', 'DT_INI_EXERC', 'DT_FIM_EXERC'])

def loadCompanyInfo():
    return loadCsv("data/cad-emp/", "cad_cia_aberta.csv")

def allHistoricalYears():
    return [2011, 2012] #2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

def loadDreItr(years):
    tempDF = pd.DataFrame() 
    for year in years:
        tempDF = pd.concat([tempDF, loadCsv("data/dre-itr/","itr_cia_aberta_DRE_con_{0}.csv".format(year))], ignore_index=True)
    return tempDF

def loadDreDfp(years):
    tempDF = pd.DataFrame() 
    for year in years:
        tempDF = pd.concat([tempDF, loadCsv("data/dre-dfp/",f'dfp_cia_aberta_DRE_con_{year}.csv')], ignore_index=True)
    return tempDF

def loadAllDreItr():
    return loadDreItr(allHistoricalYears())

def loadAllDreDfp():
    return loadDreDfp(allHistoricalYears())


def executeExperiment():
    companyInfoDF = loadCompanyInfo()
    dreItrDF = loadAllDreItr()
    dreDfpDF = loadAllDreDfp()
    print("Executando o experimento")
    print(f"Carregando Datasets. CadEmp:{companyInfoDF.shape}, DreItr:{dreItrDF.shape} e DrePfp:{dreDfpDF.shape} ")
    return companyInfoDF, dreItrDF, dreDfpDF
    
    

In [133]:
import sys, subprocess, datetime
from prov.model import ProvDocument, Namespace
from prov.dot import prov_to_dot
from IPython.display import Image

def createProvenance(agent, entity, activity, graph):
    # Creating an empty provenance document
    docProv = ProvDocument()

    # Declaring namespaces for various prefixes used in the excution of Randon Walk Experiment
    docProv.add_namespace('foaf', 'http://xmlns.com/foaf/0.1/')
    docProv.add_namespace('prov', 'http://www.w3.org/ns/prov#')
    docProv.add_namespace('void', 'http://vocab.deri.ie/void#')
    docProv.add_namespace('cvm', 'https://www.gov.br/cvm/pt-br')
    docProv.add_namespace('cvm-cademp', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2011', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2012', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2013', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2014', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2015', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2016', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2017', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2018', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2019', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2020', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2021', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-dfp-2010', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/')

    
    
    # Creating all entity
    entDsCadEmp = docProv.entity('cvm-cademp:cad_cia_aberta.csv', {'prov:label': 'Dataset com dados da empresas listadas na CVM', 'prov:type': 'void:Dataset'})
    
    entDreItr = docProv.entity('cvm:dre-trimestral', {'prov:label': 'Documrnto que representa o conceito de DREs do tipo Trimestral', 'prov:type': 'foaf:Document'})
    entDsDreItr2011 = docProv.entity("cvm-dre-itr-2011:itr_cia_aberta_2011.zip", {'prov:label': 'Dataset com DRE trimestrais, do ano de 2011', 'prov:type': 'void:Dataset'})
    docProv.wasDerivedFrom('cvm-dre-itr-2011:itr_cia_aberta_2011.zip', 'cvm:dre-trimestral')
    docProv.wasDerivedFrom('cvm-dre-itr-2011:itr_cia_aberta_DRE_con_2011.csv', 'cvm-dre-itr-2011:itr_cia_aberta_2011.zip')
    entDsDreItr2012 = docProv.entity("cvm-dre-itr-2012:itr_cia_aberta_2012.zip", {'prov:label': 'Dataset com DRE trimestrais, do ano de 2012', 'prov:type': 'void:Dataset'})
    docProv.wasDerivedFrom('cvm-dre-itr-2012:itr_cia_aberta_2012.zip', 'cvm:dre-trimestral')
    docProv.wasDerivedFrom('cvm-dre-itr-2012:itr_cia_aberta_DRE_con_2012.csv', 'cvm-dre-itr-2012:itr_cia_aberta_2012.zip')
    entDsDreItr2013 = docProv.entity("cvm-dre-itr-2013:itr_cia_aberta_2013.zip", {'prov:label': 'Dataset com DRE trimestrais, do ano de 2013', 'prov:type': 'void:Dataset'})
    docProv.wasDerivedFrom('cvm-dre-itr-2013:itr_cia_aberta_2013.zip', 'cvm:dre-trimestral')
    docProv.wasDerivedFrom('cvm-dre-itr-2013:itr_cia_aberta_DRE_con_2013.csv', 'cvm-dre-itr-2013:itr_cia_aberta_2013.zip')

    
    
    entDreDfp = docProv.entity('cvm:dre-anual', {'prov:label': 'Documento que representa o conceito de DREs do tipo Anual', 'prov:type': 'foaf:Document'})
    entDsDreDfp2010 = docProv.entity("cvm-dre-dfp-2010:dfp_cia_aberta_2010.zip", {'prov:label': 'Dataset com DRE anual, do ano de 2010', 'prov:type': 'void:Dataset'})
    docProv.wasDerivedFrom('cvm-dre-dfp-2010:dfp_cia_aberta_2010.zip', 'cvm:dre-anual')
    docProv.wasDerivedFrom('cvm-dre-dfp-2010:dfp_cia_aberta_DRE_con_2010.csv', 'cvm-dre-dfp-2010:dfp_cia_aberta_2010.zip')
    
    

    # Creating all Agents  
    agntCvm = docProv.agent("cvm:CVM", 
                            {"prov:type":"prov:Organization", "foaf:name":"Comissão de Valores Mobiliários"})
    
    # create activity of dataset creations
    actvCreateDs = docProv.activity("cvm:create-dataset")    
    
    # Generation
    docProv.wasGeneratedBy(entDsCadEmp, actvCreateDs)
    docProv.wasGeneratedBy(entDreItr, actvCreateDs)
    docProv.wasGeneratedBy(entDreDfp, actvCreateDs)
    
    docProv.wasAssociatedWith(actvCreateDs, agntCvm)

    ### END - Registering Retrospective Provenance 

    ### Optional outputs ####

    #Generating the outup - a  Provenance Graph
    dot = prov_to_dot(docProv)
    graph = graph+".png"
    dot.write_png(graph)

    #Generating the Serialization - Output XML
    docProv.serialize(entity + ".xml", format='xml') 

    #Generating the Serialization - Output Turtle
    docProv.serialize(entity + ".ttl", format='rdf', rdf_format='ttl')

In [134]:
agent    = "Grupo02-Gil-Saulo-Valquire"                                         #PROV-Agent
entity   = "DREs-CVM"                                                           #PROV-Entity
activity = "TrabalhoFinalDisciplinaFundamentoDS-2022-2"                         #PROV-Activity
graph = entity                                                                  #PROV-Graph

def main():
    if(checkingEnvironmentVersions(True)):
        executeExperiment()
        createProvenance(agent, entity, activity, graph)
        
# main() # condição suspensa para poder rodar no notebook local

# Image(graph+\".png\") # condição suspensa para poder rodar no notebook local

In [135]:
df_cia, df_itr, df_dfp = executeExperiment()

Executando o experimento
Carregando Datasets. CadEmp:(2550, 47), DreItr:(243021, 15) e DrePfp:(52046, 15) 


In [136]:
df_cia.info()
df_itr.info()
df_dfp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CNPJ_CIA            2550 non-null   object 
 1   DENOM_SOCIAL        2550 non-null   object 
 2   DENOM_COMERC        2467 non-null   object 
 3   DT_REG              2550 non-null   object 
 4   DT_CONST            1621 non-null   object 
 5   DT_CANCEL           1769 non-null   object 
 6   MOTIVO_CANCEL       1767 non-null   object 
 7   SIT                 2550 non-null   object 
 8   DT_INI_SIT          2550 non-null   object 
 9   CD_CVM              2550 non-null   int64  
 10  SETOR_ATIV          2544 non-null   object 
 11  TP_MERC             668 non-null    object 
 12  CATEG_REG           1311 non-null   object 
 13  DT_INI_CATEG        1311 non-null   object 
 14  SIT_EMISSOR         1220 non-null   object 
 15  DT_INI_SIT_EMISSOR  1220 non-null   object 
 16  CONTRO

In [137]:
# filtrar as empresas com cadastro ativo
df_cia = df_cia[df_cia['SIT'] == 'ATIVO']

In [138]:
# deletar colunas para o merge
df_cia.drop(['DENOM_SOCIAL', 'DENOM_COMERC', 'DT_REG', 'DT_CONST', 'DT_CANCEL', 'MOTIVO_CANCEL', 'SIT', 
'DT_INI_SIT', 'TP_MERC', 'CATEG_REG', 'DT_INI_CATEG', 'SIT_EMISSOR', 'DT_INI_SIT_EMISSOR', 'CONTROLE_ACIONARIO', 
'TP_ENDER', 'LOGRADOURO', 'COMPL', 'BAIRRO', 'MUN', 'UF', 'PAIS', 'CEP', 'DDD_TEL', 'TEL', 'DDD_FAX', 'FAX', 
'EMAIL', 'TP_RESP', 'RESP', 'DT_INI_RESP', 'LOGRADOURO_RESP', 'COMPL_RESP', 'BAIRRO_RESP', 'MUN_RESP', 'UF_RESP', 
'PAIS_RESP', 'CEP_RESP', 'DDD_TEL_RESP', 'TEL_RESP', 'DDD_FAX_RESP', 'FAX_RESP', 'EMAIL_RESP', 'CNPJ_AUDITOR', 
'AUDITOR'], axis=1, inplace=True)

# deletar os registros duplicados, que possuem mesmos cnpj e códigos cvm
print('Antes: Linhas = {0} | Colunas = {1}'.format(df_cia.shape[0], df_cia.shape[1]))
df_cia.drop_duplicates(subset = ['CNPJ_CIA', 'CD_CVM'], keep = 'last', inplace=True)
print('Depois: Linhas = {0} | Colunas = {1}'.format(df_cia.shape[0], df_cia.shape[1]))

df_cia.head()

Antes: Linhas = 780 | Colunas = 3
Depois: Linhas = 695 | Colunas = 3


,CNPJ_CIA,CD_CVM,SETOR_ATIV
0,08.773.135/0001-00,25224,"Construção Civil, Mat. Constr. e Decoração"
2,12.091.809/0001-55,25291,Petróleo e Gás
3,01.547.749/0001-16,16330,Emp. Adm. Part. - Energia Elétrica
4,01.851.771/0001-55,16284,Emp. Adm. Part. - Sem Setor Principal
14,07.636.657/0001-99,26425,Metalurgia e Siderurgia


In [139]:
# filtrar últimos informes trimestrais
df_itr = df_itr[df_itr['ORDEM_EXERC'] == 'ÚLTIMO']

In [140]:
# fazer merge entre cadastro e informe itr, obtendo o df itr com setor
df_itr_setor = pd.merge(df_itr, df_cia, how = 'inner', on = ['CNPJ_CIA', 'CD_CVM']).reset_index(drop=True)

# comparar registros antes e depois
print('Antes: Linhas = {0} | Colunas = {1}'.format(df_itr.shape[0], df_itr.shape[1]))
print('Depois: Linhas = {0} | Colunas = {1}'.format(df_itr_setor.shape[0], df_itr_setor.shape[1]))
print('Diferença de Linhas =', len(df_itr) - len(df_itr_setor))

df_itr_setor.head()

Antes: Linhas = 121685 | Colunas = 15
Depois: Linhas = 84550 | Colunas = 16
Diferença de Linhas = 37135


,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,SETOR_ATIV
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.01,Receita de Venda de Bens e/ou Serviços,8551982.0,S,Energia Elétrica
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-1127021.0,S,Energia Elétrica
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.02.01,Encargos Setoriais,-398353.0,N,Energia Elétrica
3,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.02.02,ICMS,-252475.0,N,Energia Elétrica
4,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.02.03,PASEP/COFINS,-476193.0,N,Energia Elétrica


In [141]:
# criar o df com informações dos trimestres 1, 2 e 3
df_trim123 = df_itr_setor.loc[lambda df_itr_setor: (df_itr_setor.DT_REFER.dt.month == 3) | 
                            ((df_itr_setor.DT_REFER.dt.month == 6) & (df_itr_setor.DT_INI_EXERC.dt.month > 3)) | 
                            ((df_itr_setor.DT_REFER.dt.month == 9) & (df_itr_setor.DT_INI_EXERC.dt.month > 6))]
df_trim123.head()                                                                                                  

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,SETOR_ATIV
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.01,Receita de Venda de Bens e/ou Serviços,8551982.0,S,Energia Elétrica
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-1127021.0,S,Energia Elétrica
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.02.01,Encargos Setoriais,-398353.0,N,Energia Elétrica
3,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.02.02,ICMS,-252475.0,N,Energia Elétrica
4,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.02.03,PASEP/COFINS,-476193.0,N,Energia Elétrica


In [142]:
df_trim123.DT_REFER.value_counts()

2012-09-30    8494
2011-09-30    8433
2012-06-30    8428
2012-03-31    8363
2011-06-30    8348
2011-03-31    8270
Name: DT_REFER, dtype: int64

In [143]:
df_trim123.DT_FIM_EXERC.value_counts()

2012-09-30    8494
2011-09-30    8433
2012-06-30    8428
2012-03-31    8363
2011-06-30    8348
2011-03-31    8270
Name: DT_FIM_EXERC, dtype: int64

In [144]:
df_trim123.describe()

,VERSAO,CD_CVM,VL_CONTA
count,50336.000000,50336.000000,5.033600e+04
mean,1.313394,14857.111610,5.725776e+04
std,0.624031,6614.676956,2.530786e+06
min,1.000000,94.000000,-1.919357e+08
25%,1.000000,9067.000000,-4.844500e+03
50%,1.000000,17329.000000,0.000000e+00
75%,1.000000,20605.000000,9.570000e+02
max,8.000000,23000.000000,2.151321e+08


In [145]:
# criar o df com informações acumuladas até o trimestre 3
df_acm3 = df_itr_setor.loc[lambda df_itr_setor: ((df_itr_setor.DT_REFER.dt.month == 9) & (df_itr_setor.DT_INI_EXERC.dt.month == 1))]
df_acm3.head()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,SETOR_ATIV
126,00.001.180/0001-26,2011-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,3.01,Receita de Venda de Bens e/ou Serviços,20689462.0,S,Energia Elétrica
128,00.001.180/0001-26,2011-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,3.02,Custo dos Bens e/ou Serviços Vendidos,0.0,S,Energia Elétrica
130,00.001.180/0001-26,2011-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,3.03,Resultado Bruto,20689462.0,S,Energia Elétrica
132,00.001.180/0001-26,2011-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,3.04,Despesas/Receitas Operacionais,-16867579.0,S,Energia Elétrica
134,00.001.180/0001-26,2011-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,3.04.01,Despesas com Vendas,0.0,S,Energia Elétrica


In [146]:
df_acm3.DT_REFER.value_counts()

2012-09-30    8360
2011-09-30    8263
Name: DT_REFER, dtype: int64

In [147]:
# filtrar últimos informes anuais
df_dfp = df_dfp[df_dfp['ORDEM_EXERC'] == 'ÚLTIMO']

In [148]:
# fazer merge entre cadastro e informe itr, criando o dataset obinvest
df_dfp_setores = pd.merge(df_dfp, df_cia, how = 'inner', on = ['CNPJ_CIA', 'CD_CVM']).reset_index(drop=True)

# comparar registros antes e depois
print('Antes: Linhas = {0} | Colunas = {1}'.format(df_dfp.shape[0], df_dfp.shape[1]))
print('Depois: Linhas = {0} | Colunas = {1}'.format(df_dfp_setores.shape[0], df_dfp_setores.shape[1]))
print('Diferença de Linhas =', len(df_dfp) - len(df_dfp_setores))

df_dfp_setores.head()

Antes: Linhas = 26093 | Colunas = 15
Depois: Linhas = 18641 | Colunas = 16
Diferença de Linhas = 7452


,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,SETOR_ATIV
0,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,3.01,Receitas da Intermediação Financeira,106919760.0,S,Bancos
1,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,3.01.01,Receita de Juros,106919760.0,S,Bancos
2,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,3.02,Despesas da Intermediação Financeira,-69976075.0,S,Bancos
3,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,3.02.01,Despesa de Juros,-61403710.0,S,Bancos
4,00.000.000/0001-91,2011-12-31,4,BCO BRASIL S.A.,1023,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,3.02.02,Despesa com Provisão para Perdas em Empréstimos,-8572365.0,S,Bancos


In [149]:
df_dfp_setores.DT_REFER.value_counts()

2012-12-31    9255
2011-12-31    9029
2011-03-31     132
2012-03-31      94
2012-06-30      66
2011-06-30      65
Name: DT_REFER, dtype: int64